In [1]:
import requests
import xlwt
from xlwt import Workbook
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import email.utils
import pandas as pd

api_url='https://remoteok.com/api/'
user_agent='Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
REQUEST_HEADER={
    'User-Agent':user_agent,
    'Accept-Language':'en-US,en;q=0.5',

}

def get_job_posting():
    res=requests.get(api_url,headers=REQUEST_HEADER)
    return res.json()

def output_job_to_excel(data):
    wb=Workbook()
    job_sheet=wb.add_sheet("Jobs")
    header=list(data[0].keys())
    for i in range(len(header)):
        job_sheet.write(0,i,header[i])
    # wb.save('remoteok_data.xls')
    for i in range(len(data)):
        dataRow=data[i]
        values=list(dataRow.values())
        for j in range(len(values)):
            job_sheet.write(i+1,j,values[j])
    wb.save('remoteok_data.xls')
    
def scrap(start,end):
    json=get_job_posting()[int(start):int(end)]
    output_job_to_excel(json)
    print(pd.DataFrame(json)['company'])

if __name__=="__main__":
    scrap(1,30)
    


0                   Sama
1     Global Elite Texas
2              Risk Labs
3               Tractian
4              Livefront
5              Deep 6 AI
6               Codility
7         Delphi Digital
8                    1kx
9               EoT Labs
10                 Eleks
11               DFINITY
12         Manta Network
13          Horizen Labs
14                 Aztec
15                 STEPN
16              OfferFit
17              BenchSci
18             Risk Labs
19               Apptegy
20                Gopuff
21                 LI.FI
22                  Enya
23             Waterfall
24            MentorMate
25              Pangolia
26                 Canva
27              Skydance
28              Skydance
Name: company, dtype: object


In [ ]:
from datetime import datetime
import requests
import csv
import bs4
import concurrent.futures
from tqdm import tqdm
USER_AGENT='Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
REQUEST_HEADER={
    'User-Agent':USER_AGENT,
    'Accept-Language':'en-US,en;q=0.5',

}
NO_THREADS=5
def get_page_html(url):
    res=requests.get(url=url,headers=REQUEST_HEADER)
    return res.content
def get_product_price(soup):
    main_price_span=soup.find('span',attrs={
        'class':'a-price-whole'
    })
    
    return main_price_span.get_text(strip=True).strip().replace('.','')
def get_product_title(soup):
    product_title=soup.find('span',id='productTitle')
    return product_title.text.strip()

def get_product_rating(soup):
    product_rating_div=soup.find('div',attrs={
        'id':'averageCustomerReviews'
    })
    product_rating_section=product_rating_div.find('span', id='acrPopover',attrs={
        'class':"reviewCountTextLinkedHistogram noUnderline"
    })

    product_rating=product_rating_section.find('span',attrs={
        'class':'a-icon-alt'
    })
    return float(product_rating.get_text(strip=True).strip().split()[0])


def extract_product_info(url,output):
    product_info={}
    html=get_page_html(url)
    soup=bs4.BeautifulSoup(html,'lxml')

    product_info['title']=get_product_title(soup)
    product_info['price']=get_product_price(soup)
    product_info['rating']=get_product_rating(soup)
    output.append( product_info)


if __name__=="__main__":
    product_data=[]
    urls=[]
    with open("amazon_web.csv",newline='')as csvFile:
        urls=list(csv.reader(csvFile,delimiter=','))
    with concurrent.futures.ThreadPoolExecutor(max_workers=NO_THREADS) as executor:
        for wkn in tqdm(range(0,len(urls))):
            executor.submit(extract_product_info,urls[wkn][0],product_data)
    output_file='amazon_product_data-{}.csv'.format(
        datetime.today().strftime("%d_%m_%Y")
    )
    with open(output_file,'w',newline='')as outputFile:
        write=csv.writer(outputFile)
        write.writerow(product_data[0].keys())
        for product in product_data:
            write.writerow(product.values())